In [1]:
import tweepy

consumer_key = "eQ8ErZL3vEvZkQvTSKMPndhzb"
consumer_secret = "ztZJo14QU1wFtREneeQ7HhBUbK2RidqIIHCvXKSZDjEqVMTprv"

access_token = "115683208-cdY9Yru9aaStNHr9QtA2uLTn4khAqpoL33HvPKgm"
access_token_secret = "4vheUcufR5nPw9WAYbMs0omnjc97KmYaN2fk3dYQ0O1UO"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)
print(api)

In [2]:
import time

def process_tweet(tweet) :
    tweet_date = tweet['created_at']
    ts = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet_date,'%a %b %d %H:%M:%S +0000 %Y'))

    tweet_id = tweet['id']
    tweet_text = tweet['text']
    if('extended_tweet' in tweet.keys()):
        tweet_text = tweet['extended_tweet']['full_text']

    user_id = tweet['user']['id']
    followers_count = tweet['user']['followers_count']
    friends_count = tweet['user']['friends_count']

    user_mentions = tweet['entities']['user_mentions']
    screen_names = [user_mention['screen_name'] for user_mention in user_mentions]
    screen_name = tweet['user']['screen_name']
    retweet_count = tweet['retweet_count']
    favorite_count = tweet['favorite_count']
    tweet_row = {'date':ts,
                 'tweet_id' : tweet_id,
                 'user_id' : user_id,
                 'followers_count' : followers_count,
                 'friends_count' : friends_count,
                 'user_mentions' : screen_names,
                 'screen_name' : screen_name,
                 'retweet_count' : retweet_count,
                 'favorite_count' : favorite_count,
                 'full_text':tweet_text}
    return tweet_row

In [3]:
#Ref : https://stackoverflow.com/questions/1060279/iterating-through-a-range-of-dates-in-python
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [4]:
#Ref : https://medium.com/@poconnell732/acquiring-free-historical-geo-located-data-from-twitter-1f8f2821e9b1
from TwitterAPI import TwitterAPI
import json
import pandas as pd
from datetime import timedelta, date

api = TwitterAPI(consumer_key=consumer_key,
                 consumer_secret=consumer_secret,
                 access_token_key=access_token,
                 access_token_secret=access_token_secret)
PRODUCT = '30day' #Using 30day API
LABEL = 'devbox2' # sandbox name
web_request_count = 0
                
start_month_date = date(2019, 6, 4)
end_month_date = date(2019, 6, 5)

for date in daterange(start_month_date, end_month_date):
    list_tweets = []
    next_token = {}
    date_str = date.strftime("%Y%m%d")
    start_date = date_str + "0000"
    end_date = date_str + "2359"
    print(start_date)
    print(end_date)
    
    while next_token is not None :
        print(next_token)
        if not next_token :
            req_dict = {'query' : '@swiggy_in OR @swiggyCares lang:en', 'fromDate': start_date, 'toDate': end_date}
        else :
            req_dict['next'] = next_token
        
        web_request_count += 1
        print('web_request_count: ', web_request_count)
        req = api.request('tweets/search/%s/:%s' % (PRODUCT, LABEL), req_dict)
        print('status_code: ', req.status_code)
        response = req.json()
        if('next' in response.keys()):
            next_token = response['next']
        else :
            next_token = None
        
        results = response['results']
        with open('swiggy/json/' + date_str + '_' + str(web_request_count) + '.json', 'w+') as f:
            json.dump(results, f)
        
        #print(results)
        for tweet in results:
            tweet_row = process_tweet(tweet)
            list_tweets.append(tweet_row)
        df = pd.DataFrame(list_tweets)
    df.to_csv('swiggy/' + date_str + '.csv', index=False)
    print(df.shape)

201906020000
201906022359
{}
web_request_count:  1
status_code:  200
eyJhdXRoZW50aWNpdHkiOiJjODY5Y2YwMjYwMzJhOWJlZGNlOTE4MWJmNzhjNTNjMDQ4ZGZkOWFjYTg1OGI5ZmQ4ZDhkZWM1ZmFkNzhjN2U3IiwiZnJvbURhdGUiOiIyMDE5MDYwMjAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYwMjIzNTkiLCJuZXh0IjoiMjAxOTA2MDIyMzU5MDAtMTEzNTI1MTY1NDk3MjkyMzkwNC0wIn0=
web_request_count:  2
status_code:  200
eyJhdXRoZW50aWNpdHkiOiI4YTAwZjVlM2Y5ZDlhNzNhNDc5MWVhN2U5NTIzMTIwNDJiMWQxMDJkZTA5YTNiMTA2MjE4NjJkZTk3OGQ5NWM5IiwiZnJvbURhdGUiOiIyMDE5MDYwMjAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYwMjIzNTkiLCJuZXh0IjoiMjAxOTA2MDIyMzU5MDAtMTEzNTIzODQyNDE1NDI1NTM2Mi0wIn0=
web_request_count:  3
status_code:  200
eyJhdXRoZW50aWNpdHkiOiI0MDk5ZTU4Yjk0YWVmZDk2ZTFmZWY0MDUzMmEwZDM1MmIxZWMzZTA2ZGZlYzBjMzdkZGViOTMzNWQwZDg2YmVjIiwiZnJvbURhdGUiOiIyMDE5MDYwMjAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYwMjIzNTkiLCJuZXh0IjoiMjAxOTA2MDIyMzU5MDAtMTEzNTIyNDAzMzAzNTY4NTg4OC0wIn0=
web_request_count:  4
status_code:  200
eyJhdXRoZW50aWNpdHkiOiIyMGE5MDc3NTQ4ZTJjYWVlMzRjOGZjY2ZjNDYxZDMwOWI4NjRiNWEzNjdmMWVmNTUz

In [6]:
#Ref : https://medium.com/@poconnell732/acquiring-free-historical-geo-located-data-from-twitter-1f8f2821e9b1
from TwitterAPI import TwitterAPI
import json
import pandas as pd
from datetime import timedelta, date

api = TwitterAPI(consumer_key=consumer_key,
                 consumer_secret=consumer_secret,
                 access_token_key=access_token,
                 access_token_secret=access_token_secret)
PRODUCT = '30day' #Using 30day API
LABEL = 'devbox2' # sandbox name
web_request_count = 0
                
start_month_date = date(2019, 6, 3)
end_month_date = date(2019, 6, 6)

for date in daterange(start_month_date, end_month_date):
    list_tweets = []
    next_token = {}
    date_str = date.strftime("%Y%m%d")
    start_date = date_str + "0000"
    end_date = date_str + "2359"
    print(start_date)
    print(end_date)
    
    while next_token is not None :
        print(next_token)
        if not next_token :
            req_dict = {'query' : '@zomatocare OR @zomatoIn lang:en', 'fromDate': start_date, 'toDate': end_date}
        else :
            req_dict['next'] = next_token
        
        web_request_count += 1
        print('web_request_count: ', web_request_count)
        req = api.request('tweets/search/%s/:%s' % (PRODUCT, LABEL), req_dict)
        print('status_code: ', req.status_code)
        response = req.json()
        if('next' in response.keys()):
            next_token = response['next']
        else :
            next_token = None
        
        results = response['results']
        with open('zomato/json/' + date_str + '_' + str(web_request_count) + '.json', 'w+') as f:
            json.dump(results, f)
        
        #print(results)
        for tweet in results:
            tweet_row = process_tweet(tweet)
            list_tweets.append(tweet_row)
        df = pd.DataFrame(list_tweets)
    df.to_csv('zomato/' + date_str + '.csv', index=False)
    print(df.shape)

201906030000
201906032359
{}
web_request_count:  1
status_code:  200
eyJhdXRoZW50aWNpdHkiOiI3MzIwODQyMzE5NTRhYmFhNWFiMzNhMzA4ZWRmOTdiNGM2YzNlYjhhNjg4OGE4OGExYzQyNWQzODgwZGU5NTJjIiwiZnJvbURhdGUiOiIyMDE5MDYwMzAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYwMzIzNTkiLCJuZXh0IjoiMjAxOTA2MDMyMzU5MDAtMTEzNTYzMDE3NzkxMDcxNDM2OC0wIn0=
web_request_count:  2
status_code:  200
eyJhdXRoZW50aWNpdHkiOiIxMWQ1NzdlYWVjNzNlYjdkOGY0MWQ3ZTZlOTYyZmQzMzI4MzUzYzAxNzBiNDIxNjdmMDQ1NTdiNzI5YThhMDVhIiwiZnJvbURhdGUiOiIyMDE5MDYwMzAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYwMzIzNTkiLCJuZXh0IjoiMjAxOTA2MDMyMzU5MDAtMTEzNTYwMzEyMzE3MzY5MTM5Mi0wIn0=
web_request_count:  3
status_code:  200
eyJhdXRoZW50aWNpdHkiOiI5OWQzNzNkYjM4YTViN2M3ZTQyNzMwMjg0MzZjZmM2YjE5ZTk1NTBhYjExNzcyMWE4MjgwZmFhZGZlNjZiNThhIiwiZnJvbURhdGUiOiIyMDE5MDYwMzAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYwMzIzNTkiLCJuZXh0IjoiMjAxOTA2MDMyMzU5MDAtMTEzNTU4NjgzNTIzMTUxODcyMC0wIn0=
web_request_count:  4
status_code:  200
eyJhdXRoZW50aWNpdHkiOiIyODBmN2M1NjFjY2RmOTgwNGE5MTRmMzMyZDk0NDZiOTkyNjUwYzUwYWYwZWE3Nzhm

status_code:  200
eyJhdXRoZW50aWNpdHkiOiI4ZDViMTQ0NTNiOTQzY2Q0NTJkZDA0M2I3ZWE1YWZkNzhiY2JiZTkxYTg4ODEzYzM0ZmIyMWIyNDMwZWRjZjhmIiwiZnJvbURhdGUiOiIyMDE5MDYwNDAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYwNDIzNTkiLCJuZXh0IjoiMjAxOTA2MDQyMzU5MDAtMTEzNTgwODg2MTU5MDI2NTg1Ni0wIn0=
web_request_count:  32
status_code:  200
eyJhdXRoZW50aWNpdHkiOiJlOWIxZjg4YWU1YWJiZjFlMWM2NWMxYjcwZGE0MmYwYTNiMTBiNWU4ODhkODUzOTI1OTYwYjVlZWVhNjY4ODE4IiwiZnJvbURhdGUiOiIyMDE5MDYwNDAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYwNDIzNTkiLCJuZXh0IjoiMjAxOTA2MDQyMzU5MDAtMTEzNTc5NzY3NDQyOTQ0NDA5Ni0wIn0=
web_request_count:  33
status_code:  200
eyJhdXRoZW50aWNpdHkiOiJkYzg5Yjc0NWY1N2NlMWU4NmRhN2VlZjcxZjY5NWY4ZTY2OTc0YzAwNTcwMTQ5OGM5OWYzNzEyMDljYmFlMmE5IiwiZnJvbURhdGUiOiIyMDE5MDYwNDAwMDAiLCJ0b0RhdGUiOiIyMDE5MDYwNDIzNTkiLCJuZXh0IjoiMjAxOTA2MDQyMzU5MDAtMTEzNTc4MDAwNjcyMDg3NjU0NC0wIn0=
web_request_count:  34
status_code:  200
eyJhdXRoZW50aWNpdHkiOiI4MTIwZWZiNTU5MjA4ZDJkOTlhMDgyODE3M2I2MjAwNDMyOGJlOWRiYzk5MDgxNDk4OWE3YzA5MDFmM2E4YjBhIiwiZnJvbURhdGUiOiIyMDE5MDYw

KeyError: 'results'